In [1]:
!pip install torch torchvision opencv-python matplotlib

In [2]:
!wget https://www.agentspace.org/download/watch-masks.zip
!unzip watch-masks.zip
!rm watch-masks.zip

--2025-10-30 07:52:44--  https://www.agentspace.org/download/watch-masks.zip
Resolving www.agentspace.org (www.agentspace.org)... 62.168.101.9
Connecting to www.agentspace.org (www.agentspace.org)|62.168.101.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1588290 (1.5M) [application/zip]
Saving to: ‘watch-masks.zip’

watch-masks.zip     100%[===================>]   1.51M  1.50MB/s    in 1.0s    

2025-10-30 07:52:47 (1.50 MB/s) - ‘watch-masks.zip’ saved [1588290/1588290]

Archive:  watch-masks.zip
   creating: watch/
   creating: watch/train/
   creating: watch/train/images/
  inflating: watch/train/images/000001.jpg  
  inflating: watch/train/images/000002.jpg  
  inflating: watch/train/images/000003.jpg  
  inflating: watch/train/images/000004.jpg  
  inflating: watch/train/images/000005.jpg  
  inflating: watch/train/images/000006.jpg  
  inflating: watch/train/images/000007.jpg  
  inflating: watch/train/images/000008.jpg  
  inflating: watch/train/image

In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T, Compose
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import numpy as np
import os, cv2

In [4]:
class CustomSegmentationDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(os.listdir(os.path.join(root, "images")))
        self.masks = sorted(os.listdir(os.path.join(root, "masks")))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])

        img = cv2.imread(img_path)[:, :, ::-1]  # BGR→RGB
        mask = cv2.imread(mask_path, 0)         # grayscale

        # get unique object ids
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]  # remove background id 0

        masks = mask == obj_ids[:, None, None]
        num_objs = len(obj_ids)

        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)  # all one class
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
            "image_id": image_id,
            "area": area,
            "iscrowd": iscrowd
        }

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)


In [6]:
def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # Replace the classifier head
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes)

    # Replace the mask head
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask, hidden_layer, num_classes)

    return model


In [13]:
# Datasets
transforms = Compose([
    ToTensor(),
    ConvertImageDtype(torch.float32),
    Resize(min_size=800, max_size=1333),
])

dataset_train = CustomSegmentationDataset('watch/train', transforms=transforms)
dataset_val = CustomSegmentationDataset('watch/val', transforms=transforms)

train_loader = DataLoader(dataset_train, batch_size=2, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

NameError: name 'Compose' is not defined

In [10]:
# Model
num_classes = 2  # background + 1 custom class
model = get_instance_segmentation_model(num_classes)
model.cuda()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:00<00:00, 205MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [11]:
# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [12]:
# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    i = 0
    for images, targets in train_loader:
        images = list(img.cuda() for img in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch}], Step [{i}], Loss: {losses.item():.4f}")
        i += 1

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-2245572285.py", line 49, in __getitem__
    img = self.transforms(img)
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchvision/transforms/transforms.py", line 137, in __call__
    return F.to_tensor(pic)
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchvision/transforms/functional.py", line 154, in to_tensor
    img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
          ^^^^^^^^

In [ ]:
# use the trained model
model.eval()
with torch.no_grad():
    img = cv2.imread("dataset/val/images/img001.jpg")[:, :, ::-1]
    img_tensor = T.ToTensor()(img).cuda()
    pred = model([img_tensor])[0]